In [1]:
import os, sys
sys.path.append(os.path.abspath("..")) # to be able to import src

In [2]:
from src.utils.logging import select_best_model

In [3]:
res = select_best_model('qm9')

2025/12/02 17:48:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/02 17:48:12 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Best overall run ID: 3353b033a87549759fbd8227ee9a1266
Best VAL_MSE: 1.7502


In [4]:
res

{'run_id': '3353b033a87549759fbd8227ee9a1266',
 'model_type': 'gcn',
 'model_uri': 'runs:/3353b033a87549759fbd8227ee9a1266/model'}

thta id corresponds to the tuning id!!!!

In [6]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()

In [7]:


def get_experiment_id_by_name(name):
    exp = client.get_experiment_by_name(name)
    if exp is None:
        raise ValueError(f"Experiment '{name}' not found.")
    return exp.experiment_id

experiment_name = "qm9"
experiment_id = get_experiment_id_by_name(experiment_name)
experiment_id


'1'

In [8]:
import mlflow
import pandas as pd

experiment_id = "1"   # <-- change to your experiment
desired_tag = res["run_id"]  # the value you want to match


df = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string=(
        "attributes.run_name = 'test_evaluation'"
        f"and tags.tuning_run_id = '{desired_tag}'"
    )
)

df


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_ev,metrics.val_mae,metrics.val_rmse,metrics.val_r2,metrics.val_mse,params.model_type,tags.mlflow.source.git.commit,tags.mlflow.runName,tags.dataset,tags.tuning_run_id,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.user,tags.data_version
0,2b9eb3bab8694542b4e24400e989f213,1,FINISHED,/home/marcos/Escritorio/AI-prod/QM9-ML/mlruns/...,2025-12-02 20:47:21.468000+00:00,2025-12-02 20:47:21.587000+00:00,0.029003,0.808414,0.956796,-0.439384,0.915458,gcn,b4af9dec94420c08dd86b131b44cbe69915345ae,test_evaluation,test,3353b033a87549759fbd8227ee9a1266,/home/marcos/Escritorio/AI-prod/QM9-ML/scripts...,LOCAL,marcos,v1


In [9]:
run_id = df.iloc[0]["run_id"]

In [10]:
run_id


run = client.get_run(run_id)

run_info = run.info            # start/end time, status, user, etc.
run_params = run.data.params   # dict
run_metrics = run.data.metrics # dict of last logged values
run_tags = run.data.tags       # dict

#print("INFO:", run_info)
print("PARAMS:", run_params)
print("METRICS:", run_metrics)
#print("TAGS:", run_tags)


PARAMS: {'model_type': 'gcn'}
METRICS: {'val_mse': 0.9154576659202576, 'val_rmse': 0.9567955136299133, 'val_mae': 0.8084142804145813, 'val_r2': -0.4393841028213501, 'val_ev': 0.029002606868743896}


# for a code snippet

In [ ]:
# load results from test
# get results from the prev. given run_id

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_id = res["run_id"]
run_id = "336b9fcac6a144ae88006d286e5439c3"

# ---- 1. RUN METADATA ----
run = client.get_run(run_id)

run_info = run.info            # start/end time, status, user, etc.
run_params = run.data.params   # dict
run_metrics = run.data.metrics # dict of last logged values
run_tags = run.data.tags       # dict

print("INFO:", run_info)
print("PARAMS:", run_params)
print("METRICS:", run_metrics)
print("TAGS:", run_tags)

"""
list and download aryifacts

artifacts = client.list_artifacts(run_id)
artifact_paths = [a.path for a in artifacts]
artifact_paths

local_dir = client.download_artifacts(run_id, "")
print("Downloaded to:", local_dir)


"""


In [ ]:
run_params

In [ ]:
run_metrics

In [ ]:
# load best model for inference

In [ ]:
"""
def get_run_data(run_id):
    client = MlflowClient()
    run = client.get_run(run_id)

    data = {
        "info": run.info,
        "params": run.data.params,
        "metrics": run.data.metrics,
        "tags": run.data.tags,
        "artifacts": {}
    }

    # download artifacts
    for a in client.list_artifacts(run_id):
        local_path = client.download_artifacts(run_id, a.path)
        data["artifacts"][a.path] = local_path

    return data

all_data = get_run_data("417c54e0be4f4b1f813036d6194144b2")
all_data
"""